# Podcasts

I will analyze a dataset of approximately 50.000 podcasts and over 1 million reviews. I have several goals: one main goal and three subgoals in various categories. 

**Main goal**

My main goal is to process and clean the data so I can use it for analysis. 
* Leave only the data I need and merging various datasets into one DataFrame 
* Ensure the data I'm using is representative 
* Ensure the data is in the right format and presented neatly

_**Genres**_

I wnat to answer to subquestions about the genres in my dataset.
1. What genres are there?
2. Which genre has the most and least podcasts?

_**Reviews**_

I want to answer one subquestion about the reviews.
1. Which genre is best reviewed, and which worst? 

_**Random Podcast Generator**_

I want to build a random podcast generator, where the user can enter a genre (if they want) and receive a random recommendation in this genre, including information on the score, reviews, and the iTunes link. 

But let's start at the beginning! Retreiving and structuring our data. 

## Finding and retrieving my data

I found the data on Kaggle. [Thought Vector offers this dataset](https://www.kaggle.com/datasets/thoughtvector/podcastreviews) with over 1 million reviews for around 50.000 podcasts, updated monthly.

The data is offered in an SQLite database. I find four datasets in this database: 

* **Reviews**: This one contains the reviews, with title, content, rating, author-id, date of writing and a podcast-id that links the sets together. 
* **Categories**: This one has the categories, or genres, linked to the podcast-id. 
* **Runs**: This one say when the data was last updated. 
* **Podcasts**: This one has the bulk of the data on the podcasts. Title, Apple iTunes-link and iTunes-ID, as well as the podcast-id. 

I exported all these as CSV-files and named them 'categories.csv', 'podcasts.csv', 'reviews.csv' and 'runs.csv'. If you try to run this notebook and download the data, please ensure you use these filenames as well.

## A first look

I'll start by importing pandas and making a DataFrame from the podcasts.csv file, to take a first look at the data.

In [ ]:
import pandas as pd

dfdata = pd.read_csv("podcasts.csv")

dfdata.head(3)

I see the Title is in the back of the DataFrame and I would like to move it to the front, so I copy the column and insert it in the front.

In [ ]:
dfdata.insert(2, 'Title', dfdata['title'])

dfdata.head(3)

I drop the old title-column and the slug-column, which I will not be using. 

In [ ]:
dfdata.drop(columns=['title', 'slug'], inplace=True)

dfdata.head(3)

## Genres

Now that the podcast-DataFrame is done, I'll start on the categories. I want to view the unique genres in this set. 

In [ ]:
dfctg = pd.read_csv("categories.csv")

dfctg['category'].unique()

Now that I know which Genres there are, I'll go back to my podcast-DataFrame. I make a copy of it, this will eventually be my main DataFrame. In the new DataFrame I'll make a copy of the podcast-id column and name it 'Genre', so I can merge it later.

In [ ]:
df = dfdata

df.insert(3, 'Genre', df['podcast_id'])

df.head(3)

I go back to my categories-DataFrame and turn it into a dictionary, with podcast-id as the key and genre as the value. I'll use the dictionary to map it into my main DataFrame later. 

In [ ]:
genredict=dict(dfctg.values)

### Merge

I'll update my main DataFrame with the genres. I'm using the dictionary to map into it and look at the result. 

In [ ]:
df['Genre'] = df['Genre'].map(genredict)
df.head(3)

There we go! To be sure, I check the unique genres again, to see if it matches my earlier analysis and to ensure there are no podcast-id's or null-values. 

In [ ]:
df['Genre'].unique()

Looking good! Finally, I don't really like that all the genres are in lower case, so I'll capitalize them. 

In [ ]:
df['Genre'] = df['Genre'].str.title()

df.head(3)

## Podcasts per genre

I'll make a graph to see the amount of podcasts per genre. 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
genrecount = df.Genre.value_counts(ascending=False)
fig, ax = plt.subplots(figsize=(12, 12))
sns.countplot(data = df, order = genrecount.index, y = "Genre", ax=ax)

### Tiny categories

I can see a few categories with barely any podcasts. I'll analyze them more closely. 

In [ ]:
print(genrecount.loc[genrecount < 500])

I decide to group the lower categories, Christianity, Islam, Judaism, Buddhism en Hinduism, into a new genre: "Religion". There's a "parent category", 'Religion-Spirituality', but also a smaller category 'Spirituality'. Therefore I think it would be better to have one category 'Religion' instead of five very small categories for each separate religion.

In [ ]:
df['Genre'] = df['Genre'].replace(['Christianity', 'Islam', 'Judaism', 'Buddhism', 'Hinduism'], 'Religion')

I'll plot the graph again, with Religion between all the other categories. 

In [ ]:
genrecount = df.Genre.value_counts(ascending=False)
fig, ax = plt.subplots(figsize=(12, 12))
sns.countplot(data = df, y = "Genre", order = genrecount.index, ax=ax)

### So many subcategories!

It's very clear there is a difference between the genres. There is a very large amount of 'Society-Culture'-podcasts, especially if we consider there are loose subgenres in the graph (such as Places-Travel and Philosophy). The same goes for Religion-Spirituality. 

I'm curious to see what happens if we add all the subgenres to the main genre. Therefore I'm making a new column in which I combine all subgenres into their corresponding main genre. 


In [ ]:
df.insert(3, 'Main-Genre', df['Genre'])

I want to have all the main genres in this column, so, I'll make a dictionary with all the current categories and their corresponding main genre. I can use this to map into my DataFrame later.

In [ ]:
#Layout: 'Current category': 'Main category'
genregroups = {
    'Society-Culture-Philosophy': 'Society-Culture',
    'Society-Culture-Places-Travel': 'Society-Culture',
    'Society-Culture-Personal-Journals': 'Society-Culture',
    'Society-Culture': 'Society-Culture',
    'Spirituality': 'Religion-Spirituality',
    'Religion': 'Religion-Spirituality',
    'Religion-Spirituality': 'Religion-Spirituality',
    'Business-Investing': 'Business',
    'Business-Careers': 'Business',
    'Business': 'Business',
    'Arts-Performing-Arts': 'Arts',
    'Arts-Design': 'Arts',
    'Arts-Visual-Arts': 'Arts',
    'Arts-Food': 'Arts',
    'Arts-Fashion-Beauty': 'Arts',
    'Arts': 'Arts',
    'Comedy': 'Comedy', 
    'Tv-Film': 'Tv-Film',
    'Education': 'Education', 
    'Music': 'Music',
    'Technology': 'Technology',
    'Kids-Family': 'Kids-Family'
}

Now that's done, I'll map it over the Main-Genre column, so that this only contains the main genres. 

In [ ]:
df['Main-Genre'] = df['Main-Genre'].map(genregroups)
df.head(3)

Now we can update the old column. First I'll change the name into "Sub-Genre". 

In [ ]:
df.rename(columns={'Genre': 'Sub-Genre'}, inplace=True)

I'll ensure only the genres with a sub-genre have data in this column. I'll go through the DataFrame and gather the indices of rows that have the same value in Main and Sub columns. 

In [ ]:
indices = []
for row in df.itertuples():
    if row[4] == row[5]:
        indices.append(row.Index)

Now I'll replace those indices with NaN in the Sub-Genre column. Only the rows with an actual Sub-Genre will have data in this column now. 

In [ ]:
import numpy as np
for i in indices:
    df.at[i, 'Sub-Genre'] = np.nan

df.head(3)

Finally, I'll make a new countplot to look at the result of only the main genres. 

In [ ]:
hoofdgenrecount = df['Main-Genre'].value_counts(ascending=False)
fig, ax = plt.subplots(figsize=(12, 12))
sns.countplot(data = df, y = "Main-Genre", order = hoofdgenrecount.index, ax=ax)

### Genres: the conclusions

In both graphs it's clear that Society-Culture is the largest category. Furthermore, in the graph with only main genres we can see this category reaches close to 10.000, which means it's around 20% of all podcasts in this dataset! 
In the split graph we can see a large number of small genres in the bottom. In the grouped graph we can see that all those are subgenres. Kids-Family is in the end the genre with the least amount of podcasts, even though in the first graph it looks to be in the middle. 

## Reviews

I'll look at my last dataset now: reviews. 

In [ ]:
revdf = pd.read_csv('reviews.csv')

revdf

### The truth behind the average grade. 

To add this set into my main DataFrame I need all the reviews grouped per podcast. I'll make a variable that I can use. Then, I'll look at the average grade per podcast (between 1 and 5). I'll add those to a dictionary to map it later, and look at the dictionary. 

In [ ]:
rating_per_podcast = revdf.groupby('podcast_id')

avrating = rating_per_podcast.mean()
avrtdict = dict(avrating.rating)

avrtdict


I notice a lot of podcasts with a round and extreme (so either 1 or 5) grade. It could be that those podcasts only receive 1's or 5's, but it's more likely that those podcasts only received a handful of reviews at most... That's why I decide to make another dictionary, namely one with the amount of reviews. I'll look for the 'count' in my grouped subset, so how many reviews each podcast has.

In [ ]:
rating_nr = rating_per_podcast.rating.count()
amountdict = dict(rating_nr)
amountdict

It's like I thought: there are a lot of podcasts with a very small number of reviews. With only 2 or 3 reviewers it's quite easy to reach a perfect 5, I imagine. To give a more clear view of our data, I'll also add a column with the amount of reviews. 

In my main DataFrame I'll copy the podcast-id column to two new ones: Average Grade and Review Amount. 

First: Average Grade. 

In [ ]:
df.insert(5, 'Avg-Grade', df['podcast_id'])

Next, 'Review-Amount'.

In [ ]:
df.insert(6, 'Review-Amount', df['podcast_id'])

Now that the columns are ready, I'll map the new data with the two dictionaries I made. I'll start with 'Avg-Grade'. 

In [ ]:
df['Avg-Grade'] = df['Avg-Grade'].map(avrtdict)

Finally 'Review-Amount',  and then I look at my result.

In [ ]:
df['Review-Amount'] = df['Review-Amount'].map(amountdict)
df.head(3)

### Old & New

Now that I saw that some podcasts only had one review, which gives a skewed view, I'm wondering if the date of the reviews matters.

Let's say that a podcast got a number of 5-star reviews, then you would assume it's a good podcast. But if this podcast is about a current topic, for example politics, and was last reviewed in 2014, it doesn't mean the podcast is still good in 2022. 

The reverse can also be true. If a podcast got a number of lower scores, but the last one is from 2016, then the podcast had 6 years to improve if it's still running. Perhaps it's really good now!

In conclusion, I think it would be useful to add the information of the date of the most recent reviews. 

I'll first look at the 'created_at' column of my review DataFrame, because the formatting seems off. I need a DateTime-format to work with it. 

In [ ]:
revdf['created_at'].info()

It's an object, and not a DateTime. I'll change it with Pandas. 

In [ ]:
revdf['created_at'] = pd.to_datetime(revdf['created_at'])

revdf['created_at'].info()

Now that it's in DateTime I can simply use my set with reviews per podcast to find the max-value. Furthermore, I'm only interested in year, month and day, and not in the time: I'll remove it from my dataset.

In [ ]:
latest_review = rating_per_podcast['created_at'].max()
latest_review = latest_review.dt.to_period('D')

The next obstacle is transporting it to my main DataFrame. I can put the values in a dictionary, but then I'll loose the DateTime value. I'd have to revert that later. There must be an easier way! 

I manage to find the merge-method, which allows me to merge two DataFrames or series on shared values - my podcast_id's. I'll also use this method to rearrange my columns, so that all the 'boring' information is at the end. 

In [ ]:
df = (df.merge(latest_review, left_on='podcast_id', right_on='podcast_id')
.reindex(columns=['Title', 'Main-Genre', 'Sub-Genre', 'Avg-Grade', 'Review-Amount', 'created_at', 'podcast_id', 'itunes_id', 'itunes_url']))

Finally I'll rename 'created_at' to 'Most-Recent', and I'll have a look at the cleaned result. 

In [ ]:
df.rename(columns={'created_at': 'Most-Recent'}, inplace=True)

df.head(3)

### Review data in figures

Now it's time to see if I can draw conclusions about the best reviewed main genre, calculating for the amount of reviews. I'll start with looking at all reviews, including those with only one or two.

First I'll make a dataset in which all the podcasts are grouped on main genre. Then I'll make a variable called 'avgenre', which means: the average grade per genre. I'll sort it from high to low. 

In [ ]:
#Making the dataset
dataset = df.groupby('Main-Genre')

#Gathering averages per genre
avgenre = dataset['Avg-Grade'].mean()
avgenre = avgenre.sort_values(ascending=False)

#Plotting the graph
fig, ax = plt.subplots(figsize=(12, 12))
avgenre.plot(kind='bar', ax=ax)

There barely seems to be a difference in grade, and each genre seems to get almost perfect scores. We however already saw that the large amount of podcasts with only five-star reviews can skew the image. That's why I want to see what happens if I leave the podcasts with smaller amount of reviews out of the view. 

I'll make a new dataset and repeat the process. 

In [ ]:
#Making the dataset
df_ten = df.loc[df['Review-Amount'] >= 10]
df_ten.rename(columns={'Avg-Grade': 'Average >= 10'}, inplace=True)
dataset_ten = df_ten.groupby('Main-Genre')

#Gathering averages per genre
avgenre_ten = dataset_ten['Average >= 10'].mean()
avgenre_ten = avgenre_ten.sort_values(ascending=False)

#Merging the datasets
df_nul_ten = pd.concat([avgenre, avgenre_ten], axis=1)

#Plotting the graph
fig, ax = plt.subplots(figsize=(12, 12))
df_nul_ten.plot(kind='bar', ax=ax, color = ['lightblue', 'darkblue'])

#Hiding copy/slice errors
pd.options.mode.chained_assignment = None

The difference is minimal. I'm curious if this is simply the case, or if there is a moment in which the amount of reviews has a clear influence on the grade of the genres. So I decide to make a very large step: gathering only the podcasts with 500 reviews or more. 

In [ ]:
#Making the dataset
df_fivehund = df.loc[df['Review-Amount'] >= 500]
df_fivehund.rename(columns={'Avg-Grade': 'Average >= 500'}, inplace=True)
dataset_fivehund = df_fivehund.groupby('Main-Genre')

#Gathering averages per genre
avgenre_fivehund = dataset_fivehund['Average >= 500'].mean()
avgenre_fivehund = avgenre_fivehund.sort_values(ascending=False)

#Merging the datasets
df_nul_ten_fifty = pd.concat([avgenre, avgenre_ten, avgenre_fivehund], axis=1)

#Plotting the graph
fig, ax = plt.subplots(figsize=(12, 12))
df_nul_ten_fifty.plot(kind='bar', ax=ax, color = ['lightblue', 'darkblue', 'black'])

Now we see extreme differences: Technology seemed to be better reviewd in the >=10 set, but plummets in the >= 500 set. Arts however seems to be better reviewed. 

I'm curious if with this dataset we have enough podcasts left, so I'll check the amount of podcasts per category. 

In [ ]:
df_fivehund['Main-Genre'].value_counts()

Ah, like I thought: by limiting at 500, we barely have any podcasts left in some categories. 

### Step by step

I'll repeat the process for several groups. 
I'll look at the main genre for podcasts with:

* All reviews
* Ten+
* Twenty-five+
* Fifty+
* Hundred+
* Fivehundred+

reviews, and I'm curious what the result is. 

I'll make new datagroups and plot one big barplot with all of them, so that I can see how the filter progresses. 

In [ ]:
#Making datasets
df_twofive = df.loc[df['Review-Amount'] >= 25]
df_twofive.rename(columns={'Avg-Grade': 'Average >= 25'}, inplace=True)
dataset_twofive = df_twofive.groupby('Main-Genre')

df_fifty = df.loc[df['Review-Amount'] >= 50]
df_fifty.rename(columns={'Avg-Grade': 'Average >= 50'}, inplace=True)
dataset_fifty = df_fifty.groupby('Main-Genre')

df_hund = df.loc[df['Review-Amount'] >= 100]
df_hund.rename(columns={'Avg-Grade': 'Average >= 100'}, inplace=True)
dataset_hund = df_hund.groupby('Main-Genre')

#Gathering averages
avgenre_fifty = dataset_fifty['Average >= 50'].mean()
avgenre_fifty = avgenre_fifty.sort_values(ascending=False)

avgenre_twofive = dataset_twofive['Average >= 25'].mean()
avgenre_twofive = avgenre_twofive.sort_values(ascending=False)

avgenre_hund = dataset_hund['Average >= 100'].mean()
avgenre_hund = avgenre_hund.sort_values(ascending=False)

#Merging datasets
avgenres = pd.concat([avgenre, avgenre_ten, avgenre_twofive, avgenre_fifty, avgenre_hund, avgenre_fivehund], axis=1)

#Making the final graph
fig, ax = plt.subplots(figsize=(25, 12))
color_dict = {'Avg-Grade':'beige', 
'Average >= 10':'#FFCDB2', 
'Average >= 25':'#FFB4A2', 
'Average >= 50':'#E5989B', 
'Average >= 100':'#B5838D', 
'Average >= 500':'#6D6875'}
column_list = ['Avg-Grade', 
'Average >= 10', 'Average >= 25', 
'Average >= 50', 'Average >= 100', 
'Average >= 500']
avgenres.plot(y=column_list, kind='bar', ax=ax, color=color_dict).legend(loc='center left',bbox_to_anchor=(1.0, 0.5));

There's a couple of genres in which the average grade stays relatively stable, no matter what the amount of reviews I filter on is. For example, Religion-Spirituality or Education.

There's also some genres in which the grade definitely drops if we leave the lower amount of reviews out of the picture: most prominently Technology, but also for example Comedy or Society-Culture.

### So - what's the best genre? 

Firstly, of all the filters, I would like to select one that seems most representative. That means: no podcasts with only a handful of reviews, but enough podcasts per genre to form a representative image. 

In the experiments above I saw that no filter, >= 10 and >= 500 don't meet this criterium. I think I'll be more succesful in the center: so, around >=25, >= 100 or >= 50. I'll look how many podcasts are left in these. 

In [ ]:
print(f"There are {len(df.index)} podcasts.\nThere are {len(df_ten.index)} podcasts with ten or more reviews.\nThere are {len(df_twofive.index)} podcasts with 25 or more reviews.\n"
f"There are {len(df_fifty.index)} podcasts with fifty or more reviews.\nThere are {len(df_hund.index)} podcasts with a hundred or more reviews.\nThere are {len(df_fivehund.index)} podcasts with fivehundred or more reviews.")

Between the filter of >= 10 and >= 100, each step, half of the podcasts disappears! I can see that with the most strict filter (>=100), still more than 1500 podcasts remain.

The most important thing is, however, that there are enough podcasts per genre to compare them. That's why I'll put them side by side in a table, so I can see the influence of each step. 

In [ ]:
countcomp = pd.DataFrame({">=25": df_twofive['Main-Genre'].value_counts(), ">=50": df_fifty['Main-Genre'].value_counts(), ">=100": df_hund['Main-Genre'].value_counts()})

countcomp

I can see that even though in the >=100 set more than 1500 podcasts remain, but some categories only have 100 or less podcasts left - 'Music' only has 49! 

In the >= 50 set each genre has at least 100 podcasts. Furthermore, each of those podcasts has 50 or more reviews. I think that's a decent amount to base an opinion off of, in other words: if 50 people think something is really good or really bad, I'd be comfortable to rely on that opinion. 

So, finally! The answer to the question: which genre is on average judged worst and best? 


In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
avgenre_fifty.plot(kind='bar', ax=ax, width=1, edgecolor='black')

It's a small victory, but 'Business' is on average judged best. The difference with 'Education' and 'Religion-Spirituality' is extremely small. 'Technology' is reviewed 'worst' on average, although 'worst' is only a relative word here: all genres are on average reviewed quite well! 

## The random-Podcast-generator!

The last goal I set for myself is building a random-podcast-generator. I would like it to be able to do the following: 
1. The user must have the choice which genre he wants to listen to (or, get a completely random recommendation) 
2. The user must have the choice which subgenre he wants to listen to (or, get a completely random recommendation within the main genre)
3. The generator must take these criteria and give back a recommendation
4. The Podcast Title has to be given
5. The Average Grade of the Podcast must be shown, as well as the amount of reviews ("This podcast received x reviews with an average grade of y")
6. The three most recent reviews must be shown, if available 
7. The iTunes-link must be shown 

### Preparation
First, I'll prepare the choice-part. The user must be able to make a choice from all main genres (or none), and then the matching subgenres if available.

I need a couple of dictionaries for this. The first one I'll make is a dictionary in which all genres are stored, with their matching subgenres. I'm looping through my DataFrame and checking if the Main-Genre with matching Sub-Genres is already in my dictionary. If not, I'll add it. Furthermore, there needs to be an (empty) category 'None', for the user that wants a random recommendation. For each Sub-Genre there also needs to be a 'None' for the same reason. 

In [ ]:
#revdf
genres = {}

for row in df.itertuples():
    hoofdgenre = row[2]
    subgenre = row[3]
    if hoofdgenre not in genres.keys():
        genres[hoofdgenre] = []
    if subgenre is not np.nan:
        if subgenre not in genres[hoofdgenre]:
            genres[hoofdgenre].append(subgenre)
        
for key in genres:
    genres[key].append('None')

genres

Now that's done, I'll make a dictionary with all the main genres assigned to a number. I'm doing this so that the user only has to type a number, and not the main genre completely correct without any spelling-errors or typo's. I'll make a list with all the unique values from the 'Main-Genre' column. Then, I'll loop through the list and add it to my dictionary with a number. 

Finally, I'm creating a new value, 'None', that I put in the top, moving 'Business' to the bottom.

In [ ]:
maingens_list = df['Main-Genre'].unique().tolist()

maingens = {}
val = 0
for genre in maingens_list:
    maingens[val] = genre
    val += 1

maingens[0] = 'None'
maingens[10] = 'Business'

maingens


The same must happen for Sub-Genres: these have to be selected by numbers as well. As an extra step, these have to stay linked to the Main-Genre, so that I only get the Sub-Genres of the selected Main-Genre. 

I'll first make an overarching dictionary - subgenredict. Within this dictionary I'll make, for each main genre, a nested dictionary, in which I put a number and then the sub genre.  

The result is a dictionary with for each main genre the matching, numbered, subgenres. 

In [ ]:
subgenredict = {}
for key in genres: 
    val = 0
    subgenredict[key] = {}
    for value in genres[key]:
        subgenredict[key][val] = value
        val += 1

subgenredict

### The result!

Now it's time to put the puzzle pieces together. The podcast generator works as follows.

#### Genre choice
Using input, the user can pick a main genre he wants to listen to. He'll type the corresponding number. The generator checks if the user selected 'None', in which case it will immediately generate a result. If the user did pick a main genre, the generator checks if there are subgenres available ánd if the user picked one. If not, then it will generate a podcast from the main genre. 

Finally, if there was a sub genre picked, the generator will give back a podcast from that subgenre. 

#### De informatie
With a print statement the basic information is displayed: Title, average grade and amount of reviews, the date of the most recent review and the iTunes-URL. 

I'm using the podcast-id to access my review-DataFrame. Then I'll sort these to the most recent date and keep the (maximum) three most recent reviews. With a loop I go through the results and print them so that they're nice and readable. 

In [ ]:
import random
Genrechoice_num = int(input(f"We'll find you a podcast. Which main genre do you pick? Type its number.\n{maingens}"))
Genrechoice = maingens[Genrechoice_num]
SubGenrechoice_num = None

#If the user chooses 'None' as genre, generate podcast immediately 
if Genrechoice == 'None':
    pick = random.randint(0, len(df.index)+1)
    result = df.iloc[[pick]]
#After the choice of a main genre, the list of subgenres is accessed 
else:
    Subgenres = subgenredict[Genrechoice]
#Check if the main genre has subgenres. If not, use the maingenre to generate a podcast 
    if len(Subgenres) <= 1:
        dfpick = df.loc[df["Main-Genre"] == Genrechoice]
        pick = random.randint(0, len(dfpick.index)+1)
        result = dfpick.iloc[[pick]]
#Otherwise, give choice for subgenre
    else:
        SubGenrechoice_num = int(input(f"Which subgenre do you want? Type its number.\n{Subgenres}"))
        SubGenrechoice = Subgenres[SubGenrechoice_num]
#If 'None' is picked for subgenre, use main genre to generate a podcast 
        if SubGenrechoice == 'None':
            dfpick = df.loc[df["Main-Genre"] == Genrechoice]
            pick = random.randint(0, len(dfpick.index)+1)
            result = dfpick.iloc[[pick]]
#Otherwise, generate podcast from the subgenre
        else:
            dfpick = df.loc[df["Sub-Genre"] == SubGenrechoice]
            pick = random.randint(0, len(dfpick.index)+1)
            result = dfpick.iloc[[pick]]

#The result is printed
print(f"Your result is: {result['Title'].values} from the genre {result['Main-Genre'].values}. This podcast received an average grade of {result['Avg-Grade'].values} from {result['Review-Amount'].values} reviews.\
The last review was on {result['Most-Recent'].values}.\nYou can find this podcast on iTunes: {result['itunes_url'].values}.")

#With the podcast_id, the maximum 3 most recent reviews are found
resultid = result['podcast_id'].to_list()
revpick = revdf.loc[revdf['podcast_id'] == resultid[0]]
reviews = revpick.sort_values('created_at', ascending=False).head(3)

#With a loop, the reviews and matching data are printed
rv = 1
print(f"These are the (maximum) 3 most recent reviews of your podcast.")
for review in reviews.itertuples():
    print(f"Review {rv}, created on {review[6]}:\n\
        {review[3]}")
    rv += 1

## That's all!

Thank you for reading and enjoy your new podcasts!